<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/person_cigarette_model_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run a model for sigarette and person detection

In [17]:
pip install ultralytics

In [18]:
from ultralytics import YOLO
import torch
from pathlib import Path
import cv2
import numpy as np
from tqdm.auto import tqdm
import os

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
class InferenceVisualizer:
    def __init__(self, model_path, conf_threshold=0.25):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Carica direttamente il modello addestrato
        self.model = YOLO(model_path)
        self.model.to(self.device)
        self.conf_threshold = conf_threshold

        # Definisci i colori per le classi (BGR per OpenCV)
        self.colors = {
            0: (0, 255, 0),    # Verde
            1: (255, 0, 0),    # Blu
            2: (0, 0, 255),    # Rosso
        }

    def process_image(self, image_path, output_path):
        """Processa una singola immagine e salva il risultato"""
        # Leggi l'immagine
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Impossibile leggere l'immagine: {image_path}")
            return

        # Esegui l'inferenza
        results = self.model.predict(image, conf=self.conf_threshold)[0]

        # Disegna le bounding box
        annotated_image = image.copy()

        # Ottieni le predizioni
        boxes = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        confidences = results.boxes.conf.cpu().numpy()

        # Disegna ogni box
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box.astype(int)
            class_id = int(cls)
            color = self.colors.get(class_id, (0, 255, 0))  # Verde di default

            # Disegna il rettangolo
            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)

            # Prepara il testo con classe e confidenza
            label = f"Class {class_id}: {conf:.2f}"

            # Calcola la posizione del testo
            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_x = x1
            text_y = y1 - 5 if y1 - 5 > text_size[1] else y1 + text_size[1]

            # Disegna uno sfondo per il testo
            cv2.rectangle(annotated_image,
                        (text_x, text_y - text_size[1] - 4),
                        (text_x + text_size[0], text_y),
                        color, -1)

            # Disegna il testo
            cv2.putText(annotated_image, label,
                       (text_x, text_y - 2),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                       (255, 255, 255), 2)

        # Salva l'immagine risultante
        cv2.imwrite(str(output_path), annotated_image)

    def process_directory(self, input_dir, output_dir):
        """Processa tutte le immagini in una directory"""
        input_path = Path(input_dir)
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        # Trova tutte le immagini
        image_files = list(input_path.glob('*.jpg')) + \
                     list(input_path.glob('*.jpeg')) + \
                     list(input_path.glob('*.png'))

        print(f"Trovate {len(image_files)} immagini")

        # Processa ogni immagine con una barra di progresso
        for img_path in tqdm(image_files, desc="Processing images"):
            output_file = output_path / f"detected_{img_path.name}"
            self.process_image(img_path, output_file)

In [21]:
def main():
    # Configura i percorsi
    model_path = '/content/drive/MyDrive/pth_person_detect/run_20250211_095730/train/weights/best.pt'  # Sostituisci con il tuo percorso
    input_dir = '/content/drive/MyDrive/Photo person/test/images'  # Sostituisci con la tua directory di input
    output_dir = '/content/drive/MyDrive/test_trained_person'  # Sostituisci con la tua directory di output

    # Crea il visualizzatore
    visualizer = InferenceVisualizer(model_path, conf_threshold=0.6)

    # Processa le immagini
    visualizer.process_directory(input_dir, output_dir)

    print("Elaborazione completata!")
    print(f"Le immagini elaborate sono state salvate in: {output_dir}")

if __name__ == "__main__":
    main()

Trovate 73 immagini


Processing images:   0%|          | 0/73 [00:00<?, ?it/s]


0: 384x640 3 nonSmokers, 1 smoker, 80.3ms
Speed: 18.6ms preprocess, 80.3ms inference, 670.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cigarette, 1 nonSmoker, 3 smokers, 7.1ms
Speed: 2.9ms preprocess, 7.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 nonSmoker, 3 smokers, 7.1ms
Speed: 4.1ms preprocess, 7.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 smoker, 8.9ms
Speed: 3.5ms preprocess, 8.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cigarette, 2 nonSmokers, 2 smokers, 6.9ms
Speed: 4.1ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 nonSmoker, 3 smokers, 6.9ms
Speed: 2.4ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 nonSmokers, 1 smoker, 11.5ms
Speed: 4.5ms preprocess, 11.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 